In [1]:
# placeholder jupyter notebook for the final project

In [2]:
import os

import pandas as pd
import numpy as np
import datetime as dt

# Assign spreadsheet filename to 'file'
Sales_file = 'Louisville Sales.xlsx'

# Load spreadsheet
salesxl = pd.ExcelFile(Sales_file)

# Print the sheet names
print(salesxl.sheet_names)

['sheet1']


In [3]:
# Load a sheet into a DataFrame by name: df1
df1 = salesxl.parse('sheet1')

# have to clean up the Age data because it is not always correct and it is being used in some decision making later

df1['Age'] = np.where(df1['Age'] > 3000, 0, df1['Age'])
df1['Age'] = np.where(df1['Age'] > 1000, 2020 - df1['Age'].astype(int), df1['Age'])
df1['Age'] = np.where(df1['Age'] > 400, 2020 - df1['Year Built'].astype(int), df1['Age'])
df1['Age'] = np.where(df1['Age'] > 400, 0, df1['Age'])

# testing my cleanup with some specific errorous lines
#df1.loc[4947,'Age']
#df1.loc[160,'Age']
#df1.loc[2098,'Age']
#df1.loc[1918,'Age']



In [4]:
Rent_file = 'Rental data.xlsx'
rentxl = pd.ExcelFile(Rent_file)
print(rentxl.sheet_names)

['sheet1']


In [5]:
df2 = rentxl.parse('sheet1')

# needed to clean up the Rental Age data as well from wrong inputs

df2['Age'] = np.where(df2['Age'] > 3000, 0, df2['Age'])
df2['Age'] = np.where(df2['Age'] > 1800, 2020 - df2['Age'].astype(int), df2['Age'])
df2['Age'] = np.where(df2['Age'] > 400, 0, df2['Age'])

#df2.loc[141,'Age']
#df2.loc[18,'Age']
#df2.loc[56,'Age']
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3887 entries, 0 to 3886
Data columns (total 55 columns):
List Number                             3887 non-null int64
Book Section                            3887 non-null object
Property Sub Type                       3887 non-null object
Listing Date                            3887 non-null datetime64[ns]
Rented Date                             3750 non-null datetime64[ns]
Status Change Date                      3887 non-null datetime64[ns]
Original List Price                     3887 non-null float64
Rent/Month                              3887 non-null float64
Rented Price                            641 non-null float64
Area                                    3887 non-null int64
Street #                                3887 non-null object
St Dir                                  445 non-null object
Street                                  3887 non-null object
Unit #                                  974 non-null object
St Suffix        

In [6]:
# need to drop rental data that is from before 2019 to make sure it is relevant
df2 = df2[(df2['Status Change Date'].dt.year >= 2019)]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1804 entries, 6 to 3886
Data columns (total 55 columns):
List Number                             1804 non-null int64
Book Section                            1804 non-null object
Property Sub Type                       1804 non-null object
Listing Date                            1804 non-null datetime64[ns]
Rented Date                             1667 non-null datetime64[ns]
Status Change Date                      1804 non-null datetime64[ns]
Original List Price                     1804 non-null float64
Rent/Month                              1804 non-null float64
Rented Price                            641 non-null float64
Area                                    1804 non-null int64
Street #                                1804 non-null object
St Dir                                  212 non-null object
Street                                  1804 non-null object
Unit #                                  483 non-null object
St Suffix        

In [7]:
df1.head()

,List Number,Book Section,Property Sub Type,Listing Date,Sold Date,Contract Acceptance Date,Original List Price,Listing Price,Sold Price,Area,...,End Date,HOA Y/N,Stories,# Upper Floor Bedrooms,First Floor Laundry,Monthly Maintenance,Seller Concessions,HOA Annual $,Closing Costs and Prepaids,Days on Market
0,1573919,Single Family Residence,SF,2020-11-12,2020-12-30,2020-12-18,189950.0,189950.0,189900.0,5,...,2021-05-12,Y,1,0,N,NaN,Y,400.0,4600.0,36
1,1572930,Single Family Residence,SF,2020-10-30,2020-12-31,2020-11-13,184900.0,182900.0,175000.0,5,...,2021-10-30,N,1,0,Y,NaN,N,0.0,NaN,14
2,1574676,Single Family Residence,SF,2020-11-02,2020-11-20,2020-11-02,290000.0,290000.0,290000.0,9,...,2020-11-30,N,1,0,N,NaN,N,0.0,NaN,0
3,1574083,Condominium,CP,2020-11-16,2021-01-08,2020-12-21,69900.0,67000.0,62000.0,3,...,2021-02-15,Y,1,0,N,NaN,N,3153.0,NaN,35
4,1572788,Condominium,CP,2020-10-29,2020-11-30,2020-10-30,115000.0,115000.0,115000.0,7,...,2021-04-29,Y,1,0,Y,NaN,N,2328.0,NaN,1


In [8]:
# calculating subdivision level average price per sqft and average age and average size for both rental and sales

groupped_sales_df = df1.groupby('Subdivision/Condo')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "Sold Price/SqFt": "Average_Sold_Price", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})

sales_count = df1.groupby('Subdivision/Condo')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='sales_count')
groupped_sales_df = pd.merge(groupped_sales_df, sales_count, on="Subdivision/Condo")
groupped_sales_df

,Subdivision/Condo,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count
0,1400 Willow,335.096667,40.000000,2584.666667,3
1,2000 SPRING CONDOMIN,51.030000,64.000000,1113.000000,1
2,550 LOFTS,189.930000,97.000000,1211.000000,1
3,ABBEY WOODS,131.650000,24.500000,1871.500000,2
4,ABERDEEN,208.615714,79.214286,2469.500000,14
5,ACADEMY ESTATES,189.814000,6.400000,3964.800000,5
6,ACADEMY RIDGE,144.336667,16.666667,2128.333333,3
7,ACADEMY RIDGE AT LANDIS LAKE,145.187500,17.250000,2661.500000,4
8,ADAMS HILL,102.107500,41.750000,1174.000000,4
9,ADAMS RUN,117.815000,18.045455,2193.500000,22


In [9]:
groupped_rental_df = df2.groupby('Subdivision/Condo')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "List Price/SqFt": "Average_Rent_Price", 
    "Age": "Average_Rent_Age", 
    "SqFt -Total Finished": "Average_Rent_SqFt"
})


rent_count = df2.groupby('Subdivision/Condo')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='rent_count')

groupped_rental_df = pd.merge(groupped_rental_df, rent_count, on="Subdivision/Condo")

groupped_rental_df

,Subdivision/Condo,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count
0,2000 SPRING CONDOMIN,1.030000,102.000000,1600.000000,1
1,ABERDEEN,1.090000,77.000000,2641.000000,1
2,ACADEMY RIDGE,0.925000,17.500000,2058.000000,2
3,ACADEMY RIDGE AT LANDIS LAKE,1.000000,17.000000,2030.000000,1
4,ADAMS RUN,1.000000,19.500000,1699.000000,2
5,AIRVIEW,0.955000,64.000000,847.500000,2
6,ALHAMBRA HEIGHTS,1.300000,71.000000,1500.000000,1
7,ALIA,0.880000,21.000000,5030.000000,1
8,ALPHA GARDENS,0.760000,52.000000,2495.000000,1
9,ALTA VISTA,1.210000,50.000000,4789.000000,1


In [10]:
# Merge the 2 dataframes and only keep the subdivisions where there is both sales and rental data available 

subdivision_df = pd.merge(groupped_sales_df, groupped_rental_df, on="Subdivision/Condo", how='inner')
subdivision_df

,Subdivision/Condo,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count
0,2000 SPRING CONDOMIN,51.030000,64.000000,1113.000000,1,1.030000,102.000000,1600.000000,1
1,ABERDEEN,208.615714,79.214286,2469.500000,14,1.090000,77.000000,2641.000000,1
2,ACADEMY RIDGE,144.336667,16.666667,2128.333333,3,0.925000,17.500000,2058.000000,2
3,ACADEMY RIDGE AT LANDIS LAKE,145.187500,17.250000,2661.500000,4,1.000000,17.000000,2030.000000,1
4,ADAMS RUN,117.815000,18.045455,2193.500000,22,1.000000,19.500000,1699.000000,2
5,AIRVIEW,154.500000,70.000000,1255.000000,1,0.955000,64.000000,847.500000,2
6,ALHAMBRA HEIGHTS,201.675714,74.285714,2480.142857,7,1.300000,71.000000,1500.000000,1
7,ALPHA GARDENS,103.210000,49.000000,1182.000000,1,0.760000,52.000000,2495.000000,1
8,ALTA VISTA,153.835000,46.000000,5449.000000,2,1.210000,50.000000,4789.000000,1
9,ANCHORAGE,206.417586,63.793103,4508.310345,29,1.240000,56.000000,2700.000000,1


In [11]:
subdivision_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482 entries, 0 to 481
Data columns (total 9 columns):
Subdivision/Condo     482 non-null object
Average_Sold_Price    482 non-null float64
Average_Sold_Age      482 non-null float64
Average_Sold_SqFt     482 non-null float64
sales_count           482 non-null int64
Average_Rent_Price    480 non-null float64
Average_Rent_Age      482 non-null float64
Average_Rent_SqFt     482 non-null float64
rent_count            482 non-null int64
dtypes: float64(6), int64(2), object(1)
memory usage: 37.7+ KB


In [12]:
subdivision_df['rent_sales_ratio'] = pd.eval('subdivision_df.Average_Rent_Price/subdivision_df.Average_Sold_Price')*1000
subdivision_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482 entries, 0 to 481
Data columns (total 10 columns):
Subdivision/Condo     482 non-null object
Average_Sold_Price    482 non-null float64
Average_Sold_Age      482 non-null float64
Average_Sold_SqFt     482 non-null float64
sales_count           482 non-null int64
Average_Rent_Price    480 non-null float64
Average_Rent_Age      482 non-null float64
Average_Rent_SqFt     482 non-null float64
rent_count            482 non-null int64
rent_sales_ratio      480 non-null float64
dtypes: float64(7), int64(2), object(1)
memory usage: 41.4+ KB


In [13]:
# exclude items from the dataframe where the average SqFt and/or Age of the houses sold are very different (more than 50% higher
# or lower) from the average age and/of SqFt of the rented homes -> the Rental and Sales price comparison is not relevant

subdivision_df['rent_sales_age_percent'] = pd.eval('abs((subdivision_df.Average_Sold_Age-subdivision_df.Average_Rent_Age)/subdivision_df.Average_Sold_Age)')
subdivision_df['rent_sales_SqFt_percent'] = pd.eval('abs((subdivision_df.Average_Sold_SqFt-subdivision_df.Average_Rent_SqFt)/subdivision_df.Average_Sold_SqFt)')
subdivision1_df = subdivision_df.drop(subdivision_df[subdivision_df['rent_sales_age_percent'] > .5].index)
reduced_subdivision_df = subdivision1_df.drop(subdivision1_df[subdivision1_df['rent_sales_SqFt_percent'] > .5].index)
reduced_subdivision_df

,Subdivision/Condo,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count,rent_sales_ratio,rent_sales_age_percent,rent_sales_SqFt_percent
1,ABERDEEN,208.615714,79.214286,2469.500000,14,1.090000,77.000000,2641.000000,1,5.224918,0.027953,0.069447
2,ACADEMY RIDGE,144.336667,16.666667,2128.333333,3,0.925000,17.500000,2058.000000,2,6.408628,0.050000,0.033046
3,ACADEMY RIDGE AT LANDIS LAKE,145.187500,17.250000,2661.500000,4,1.000000,17.000000,2030.000000,1,6.887645,0.014493,0.237272
4,ADAMS RUN,117.815000,18.045455,2193.500000,22,1.000000,19.500000,1699.000000,2,8.487884,0.080605,0.225439
5,AIRVIEW,154.500000,70.000000,1255.000000,1,0.955000,64.000000,847.500000,2,6.181230,0.085714,0.324701
6,ALHAMBRA HEIGHTS,201.675714,74.285714,2480.142857,7,1.300000,71.000000,1500.000000,1,6.445992,0.044231,0.395196
8,ALTA VISTA,153.835000,46.000000,5449.000000,2,1.210000,50.000000,4789.000000,1,7.865570,0.086957,0.121123
9,ANCHORAGE,206.417586,63.793103,4508.310345,29,1.240000,56.000000,2700.000000,1,6.007240,0.122162,0.401106
10,ANCHORAGE PIKE,156.790000,33.000000,1722.000000,1,1.240000,34.000000,1676.000000,3,7.908668,0.030303,0.026713
11,APPLE VALLEY,142.089231,34.846154,2302.307692,13,0.891667,33.000000,1401.833333,6,6.275399,0.052980,0.391118


In [14]:
# show the subdivisions with the highest rent(per sqft)/sales(per sqft) ratio -> where you get the most bang for your buck
# if investing in a rental property

sorted_subdivision_df = reduced_subdivision_df.sort_values(['rent_sales_ratio'], ascending=False)
sorted_subdivision_df

,Subdivision/Condo,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count,rent_sales_ratio,rent_sales_age_percent,rent_sales_SqFt_percent
78,CALIFORNIA,27.542500,83.750000,1151.500000,4,0.870000,110.000000,954.000000,1,31.587547,0.313433,0.171515
242,INNWOOD,51.577500,39.000000,538.000000,4,1.250000,38.000000,538.000000,1,24.235374,0.025641,0.000000
442,WESTMEADE,29.990000,91.000000,1142.000000,1,0.720000,94.000000,1040.000000,1,24.008003,0.032967,0.089317
107,CHURCHILL,61.666667,97.333333,958.333333,3,1.330000,60.000000,500.000000,1,21.567568,0.383562,0.478261
335,PORTLAND,35.931905,110.454545,1210.272727,22,0.693333,118.000000,1108.666667,3,19.295758,0.068313,0.083953
138,DEARBORN,52.955000,71.500000,1205.000000,4,0.890000,83.000000,1091.500000,2,16.806723,0.160839,0.094191
264,LENOX,73.103333,76.666667,940.933333,15,1.220000,69.000000,572.000000,1,16.688705,0.100000,0.392093
108,CHURCHILL EAST,38.630000,90.500000,1034.000000,2,0.640000,99.000000,1400.000000,1,16.567435,0.093923,0.353965
48,BOULEVARD HEIGHTS,72.772000,80.000000,852.400000,5,1.200000,92.000000,1278.000000,1,16.489859,0.150000,0.499296
146,DULWORTH JC,72.133333,66.666667,1187.333333,3,1.120000,61.000000,1114.000000,1,15.526802,0.085000,0.061763


*Calculate the same for zip codes*

In [15]:
# creating the calculated file for the Sales data, same as above, but now the groupping is based on zip code

groupped_zip_sales_df = df1.groupby('Zip Code')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "Sold Price/SqFt": "Average_Sold_Price", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})
sales_count = df1.groupby('Zip Code')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='sales_count')
groupped_zip_sales_df = pd.merge(groupped_zip_sales_df, sales_count, on="Zip Code")
groupped_zip_sales_df

,Zip Code,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count
0,40023,185.461216,12.945946,3342.162162,74
1,40025,215.135000,18.500000,8858.500000,2
2,40027,182.070000,45.000000,5284.000000,1
3,40056,177.560000,39.000000,1025.000000,1
4,40059,198.714271,20.653247,3353.501299,385
5,40118,121.297800,53.792079,1431.297030,101
6,40177,100.000000,42.000000,2200.000000,1
7,40202,174.878000,86.600000,1340.080000,50
8,40203,107.510174,97.860465,1647.970930,172
9,40204,176.563095,99.276353,1798.578348,351


In [16]:
# same groupped, calculated file for the Rental data based on zip code

groupped_zip_rental_df = df2.groupby('Zip Code')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "List Price/SqFt": "Average_Rent_Price", 
    "Age": "Average_Rent_Age", 
    "SqFt -Total Finished": "Average_Rent_SqFt"
})


rent_count = df2.groupby('Zip Code')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='rent_count')

groupped_zip_rental_df = pd.merge(groupped_zip_rental_df, rent_count, on="Zip Code")

groupped_zip_rental_df

,Zip Code,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count
0,40023,0.900000,23.000000,2299.400000,5
1,40059,1.177031,16.846154,2223.307692,65
2,40118,1.095000,26.777778,1483.777778,9
3,40155,1.190000,61.500000,550.000000,2
4,40165,1.100000,12.000000,1094.000000,1
5,40202,1.423636,52.227273,1289.363636,22
6,40203,0.933800,94.129630,1262.500000,54
7,40204,2.399145,85.155738,1146.245902,122
8,40205,1.128437,79.367347,1585.448980,98
9,40206,1.167857,59.544974,1016.126984,189


In [17]:
# Merge the 2 dataframes and only keep the Zip Codes where there is both sales and rental data available 

zip_df = pd.merge(groupped_zip_sales_df, groupped_zip_rental_df, on="Zip Code", how='inner')
zip_df

,Zip Code,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count
0,40023,185.461216,12.945946,3342.162162,74,0.900000,23.000000,2299.400000,5
1,40059,198.714271,20.653247,3353.501299,385,1.177031,16.846154,2223.307692,65
2,40118,121.297800,53.792079,1431.297030,101,1.095000,26.777778,1483.777778,9
3,40202,174.878000,86.600000,1340.080000,50,1.423636,52.227273,1289.363636,22
4,40203,107.510174,97.860465,1647.970930,172,0.933800,94.129630,1262.500000,54
5,40204,176.563095,99.276353,1798.578348,351,2.399145,85.155738,1146.245902,122
6,40205,183.054795,75.050000,2154.856818,440,1.128437,79.367347,1585.448980,98
7,40206,169.943024,74.846966,1740.155673,379,1.167857,59.544974,1016.126984,189
8,40207,191.424633,63.482428,2469.720447,626,1.144745,61.900000,1590.742857,140
9,40208,91.341953,99.653846,1595.000000,130,0.925500,97.850000,1484.700000,40


In [18]:
zip_df['rent_sales_ratio'] = pd.eval('zip_df.Average_Rent_Price/zip_df.Average_Sold_Price')*1000
zip_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 33
Data columns (total 10 columns):
Zip Code              34 non-null int64
Average_Sold_Price    34 non-null float64
Average_Sold_Age      34 non-null float64
Average_Sold_SqFt     34 non-null float64
sales_count           34 non-null int64
Average_Rent_Price    34 non-null float64
Average_Rent_Age      34 non-null float64
Average_Rent_SqFt     34 non-null float64
rent_count            34 non-null int64
rent_sales_ratio      34 non-null float64
dtypes: float64(7), int64(3)
memory usage: 2.9 KB


In [19]:
# exclude items from the dataframe where the average SqFt and/or Age of the houses sold are very different (more than 50% higher
# or lower) from the average age and/of SqFt of the rented homes -> the Rental and Sales price comparison is not relevant

zip_df['rent_sales_age_percent'] = pd.eval('abs((zip_df.Average_Sold_Age-zip_df.Average_Rent_Age)/zip_df.Average_Sold_Age)')
zip_df['rent_sales_SqFt_percent'] = pd.eval('abs((zip_df.Average_Sold_SqFt-zip_df.Average_Rent_SqFt)/zip_df.Average_Sold_SqFt)')
zip1_df = zip_df.drop(zip_df[zip_df['rent_sales_age_percent'] > .5].index)
reduced_zip_df = zip1_df.drop(zip1_df[zip1_df['rent_sales_SqFt_percent'] > .5].index)
reduced_zip_df

,Zip Code,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count,rent_sales_ratio,rent_sales_age_percent,rent_sales_SqFt_percent
1,40059,198.714271,20.653247,3353.501299,385,1.177031,16.846154,2223.307692,65,5.923235,0.184334,0.337019
3,40202,174.878000,86.600000,1340.080000,50,1.423636,52.227273,1289.363636,22,8.140740,0.396914,0.037846
4,40203,107.510174,97.860465,1647.970930,172,0.933800,94.129630,1262.500000,54,8.685690,0.038124,0.233906
5,40204,176.563095,99.276353,1798.578348,351,2.399145,85.155738,1146.245902,122,13.588034,0.142235,0.362693
6,40205,183.054795,75.050000,2154.856818,440,1.128437,79.367347,1585.448980,98,6.164479,0.057526,0.264244
7,40206,169.943024,74.846966,1740.155673,379,1.167857,59.544974,1016.126984,189,6.872051,0.204444,0.416071
8,40207,191.424633,63.482428,2469.720447,626,1.144745,61.900000,1590.742857,140,5.980132,0.024927,0.355902
9,40208,91.341953,99.653846,1595.000000,130,0.925500,97.850000,1484.700000,40,10.132255,0.018101,0.069154
10,40209,60.878571,99.714286,1045.000000,7,1.020000,119.000000,686.000000,1,16.754664,0.193410,0.343541
11,40210,52.298871,89.579365,1243.150794,126,0.733333,92.888889,1255.555556,9,14.021973,0.036945,0.009978


In [20]:
# show the subdivisions with the highest rent(per sqft)/sales(per sqft) ratio -> where you get the most bang for your buck
# if investing in a rental property

sorted_zip_df = reduced_zip_df.sort_values(['rent_sales_ratio'], ascending=False)
sorted_zip_df

,Zip Code,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count,rent_sales_ratio,rent_sales_age_percent,rent_sales_SqFt_percent
10,40209,60.878571,99.714286,1045.000000,7,1.020000,119.000000,686.000000,1,16.754664,0.193410,0.343541
13,40212,45.904387,99.948718,1403.352564,156,0.728000,87.705882,1174.588235,17,15.859051,0.122491,0.163013
11,40210,52.298871,89.579365,1243.150794,126,0.733333,92.888889,1255.555556,9,14.021973,0.036945,0.009978
5,40204,176.563095,99.276353,1798.578348,351,2.399145,85.155738,1146.245902,122,13.588034,0.142235,0.362693
12,40211,53.871957,80.457447,1432.861702,188,0.700833,81.375000,1341.083333,24,13.009242,0.011404,0.064052
16,40215,88.481913,79.524638,1144.014493,345,0.928780,76.073171,941.024390,41,10.496840,0.043401,0.177437
9,40208,91.341953,99.653846,1595.000000,130,0.925500,97.850000,1484.700000,40,10.132255,0.018101,0.069154
4,40203,107.510174,97.860465,1647.970930,172,0.933800,94.129630,1262.500000,54,8.685690,0.038124,0.233906
17,40216,119.521302,56.898734,1596.825949,632,1.004545,54.963636,1287.236364,55,8.404740,0.034010,0.193878
31,40272,124.015239,46.822804,1615.178737,649,1.012000,48.400000,1296.400000,40,8.160287,0.033684,0.197364


We have selected the top zip codes to invest in based on average Sales price and Rental income

Next, in the original dataframes I will create tiers based on SqFt to be able to demonstrate the connection between relative house price to monthly rental income for houses of various sizes in the same zip code (what is the cost of entry and the expected rental income per month)

1. I will create tiers based on size
2. I will add a new column to each original dataframes that adds the tiers for each entry
3. I will group them by zip code and by size tier
4. I will merge the 2 databases
5. I will visualize the outcome for the top 5 zip codes


In [21]:
conditions_Sales = [
    (df1['SqFt -Total Finished'] <= 1000),
    (df1['SqFt -Total Finished'] > 1000) & (df1['SqFt -Total Finished'] <= 2000),
    (df1['SqFt -Total Finished'] > 2000) & (df1['SqFt -Total Finished'] <= 3000),
    (df1['SqFt -Total Finished'] > 3000) & (df1['SqFt -Total Finished'] <= 4000),
    (df1['SqFt -Total Finished'] > 4000)
]

conditions_Rental = [
    (df2['SqFt -Total Finished'] <= 1000),
    (df2['SqFt -Total Finished'] > 1000) & (df2['SqFt -Total Finished'] <= 2000),
    (df2['SqFt -Total Finished'] > 2000) & (df2['SqFt -Total Finished'] <= 3000),
    (df2['SqFt -Total Finished'] > 3000) & (df2['SqFt -Total Finished'] <= 4000),
    (df2['SqFt -Total Finished'] > 4000)
]

values = ['0 - 1000', '1000 - 2000', '2000 - 3000', '3000 - 4000', '4000+',]

df1['size group'] = np.select(conditions_Sales, values)
df2['size group'] = np.select(conditions_Rental, values)

df1

,List Number,Book Section,Property Sub Type,Listing Date,Sold Date,Contract Acceptance Date,Original List Price,Listing Price,Sold Price,Area,...,HOA Y/N,Stories,# Upper Floor Bedrooms,First Floor Laundry,Monthly Maintenance,Seller Concessions,HOA Annual $,Closing Costs and Prepaids,Days on Market,size group
0,1573919,Single Family Residence,SF,2020-11-12,2020-12-30,2020-12-18,189950.0,189950.0,189900.0,5,...,Y,1,0,N,NaN,Y,400.0,4600.0,36,1000 - 2000
1,1572930,Single Family Residence,SF,2020-10-30,2020-12-31,2020-11-13,184900.0,182900.0,175000.0,5,...,N,1,0,Y,NaN,N,0.0,NaN,14,2000 - 3000
2,1574676,Single Family Residence,SF,2020-11-02,2020-11-20,2020-11-02,290000.0,290000.0,290000.0,9,...,N,1,0,N,NaN,N,0.0,NaN,0,2000 - 3000
3,1574083,Condominium,CP,2020-11-16,2021-01-08,2020-12-21,69900.0,67000.0,62000.0,3,...,Y,1,0,N,NaN,N,3153.0,NaN,35,0 - 1000
4,1572788,Condominium,CP,2020-10-29,2020-11-30,2020-10-30,115000.0,115000.0,115000.0,7,...,Y,1,0,Y,NaN,N,2328.0,NaN,1,1000 - 2000
5,1570821,Single Family Residence,SF,2020-10-02,2020-12-30,2020-12-18,724950.0,665000.0,650000.0,8,...,Y,1,0,Y,NaN,N,600.0,NaN,77,4000+
6,1574581,Single Family Residence,SF,2020-11-23,2021-01-08,2020-11-29,264900.0,264900.0,269900.0,6,...,Y,1,0,Y,NaN,Y,150.0,5000.0,6,2000 - 3000
7,1568867,Single Family Residence,SF,2020-09-10,2020-11-06,2020-09-10,150000.0,150000.0,152000.0,6,...,N,2,3,N,NaN,Y,0.0,5000.0,0,1000 - 2000
8,1567169,Single Family Residence,SF,2020-08-20,2020-10-30,2020-08-20,424000.0,424000.0,424000.0,8,...,Y,1,0,Y,NaN,N,175.0,NaN,0,2000 - 3000
9,1573892,Single Family Residence,SF,2020-11-12,2021-01-08,2020-12-02,165000.0,155000.0,158000.0,4,...,N,1,0,N,NaN,Y,0.0,3013.0,20,1000 - 2000


In [22]:
groupped_zip_sales_size_df = df1.groupby(['Zip Code', 'size group'])['Sold Price', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "Sold Price": "Average_Sold_Price", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})
sales_count = df1.groupby(['Zip Code', 'size group'])['Sold Price', 'Age', 'SqFt -Total Finished'].size().reset_index(name='sales_count')
groupped_zip_sales_size_df = pd.merge(groupped_zip_sales_size_df, sales_count, on=['Zip Code', 'size group'])
# format the numbers in the table
pd.options.display.float_format = '{:,.0f}'.format
groupped_zip_sales_size_df

,Zip Code,size group,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count
0,40023,0 - 1000,"83,990",48,800,1
1,40023,1000 - 2000,"337,464",13,"1,734",10
2,40023,2000 - 3000,"363,143",15,"2,530",20
3,40023,3000 - 4000,"455,971",9,"3,447",22
4,40023,4000+,"568,438",13,"4,893",21
5,40025,4000+,"1,250,500",18,"8,858",2
6,40027,4000+,"800,000",45,"5,284",1
7,40056,1000 - 2000,"182,000",39,"1,025",1
8,40059,0 - 1000,"141,312",28,782,12
9,40059,1000 - 2000,"295,582",19,"1,585",59


In [23]:
groupped_zip_rental_size_df = df2.groupby(['Zip Code', 'size group'])['Rent/Month', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "Rent/Month": "Average_Monthly_Rent", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})
sales_count = df2.groupby(['Zip Code', 'size group'])['Rent/Month', 'Age', 'SqFt -Total Finished'].size().reset_index(name='sales_count')
groupped_zip_rental_size_df = pd.merge(groupped_zip_rental_size_df, sales_count, on=['Zip Code', 'size group'])
# format the numbers in the table
pd.options.display.float_format = '{:,.0f}'.format
groupped_zip_rental_size_df

,Zip Code,size group,Average_Monthly_Rent,Average_Sold_Age,Average_Sold_SqFt,sales_count
0,40023,1000 - 2000,"1,295",5,"1,219",2
1,40023,2000 - 3000,"2,245",50,"3,000",2
2,40023,3000 - 4000,"2,700",5,"3,059",1
3,40059,0 - 1000,"1,342",22,904,3
4,40059,1000 - 2000,"2,091",11,"1,618",26
5,40059,2000 - 3000,"2,468",13,"2,364",27
6,40059,3000 - 4000,"2,342",38,"3,409",6
7,40059,4000+,"4,667",54,"5,153",3
8,40118,1000 - 2000,"1,545",27,"1,484",9
9,40155,0 - 1000,650,62,550,2


In [24]:
zip_size_group_df = pd.merge(groupped_zip_sales_size_df, groupped_zip_rental_size_df, on=['Zip Code', 'size group'], how='inner')
zip_size_group_df

,Zip Code,size group,Average_Sold_Price,Average_Sold_Age_x,Average_Sold_SqFt_x,sales_count_x,Average_Monthly_Rent,Average_Sold_Age_y,Average_Sold_SqFt_y,sales_count_y
0,40023,1000 - 2000,"337,464",13,"1,734",10,"1,295",5,"1,219",2
1,40023,2000 - 3000,"363,143",15,"2,530",20,"2,245",50,"3,000",2
2,40023,3000 - 4000,"455,971",9,"3,447",22,"2,700",5,"3,059",1
3,40059,0 - 1000,"141,312",28,782,12,"1,342",22,904,3
4,40059,1000 - 2000,"295,582",19,"1,585",59,"2,091",11,"1,618",26
5,40059,2000 - 3000,"448,494",18,"2,488",117,"2,468",13,"2,364",27
6,40059,3000 - 4000,"532,574",22,"3,463",79,"2,342",38,"3,409",6
7,40059,4000+,"706,721",22,"5,284",118,"4,667",54,"5,153",3
8,40118,1000 - 2000,"147,064",53,"1,369",61,"1,545",27,"1,484",9
9,40202,0 - 1000,"132,580",98,796,15,"1,005",94,770,5


In [25]:
sorted_zip_df = sorted_zip_df.reset_index(drop=True)
sorted_zip_df.head()

,Zip Code,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count,rent_sales_ratio,rent_sales_age_percent,rent_sales_SqFt_percent
0,40209,61,100,"1,045",7,1,119,686,1,17,0,0
1,40212,46,100,"1,403",156,1,88,"1,175",17,16,0,0
2,40210,52,90,"1,243",126,1,93,"1,256",9,14,0,0
3,40204,177,99,"1,799",351,2,85,"1,146",122,14,0,0
4,40211,54,80,"1,433",188,1,81,"1,341",24,13,0,0


In [26]:
top_5_results = sorted_zip_df.loc[0:4, 'Zip Code']
print(top_5_results)

0    40209
1    40212
2    40210
3    40204
4    40211
Name: Zip Code, dtype: int64
